# Generator opisów oraz pytań na Bieg Nocny
Przy urzyciu tego notatnika możesz wygenerować sobie opisy i pytania na punkty których używamy na biegu nocnym. Program opiera się na automatycznym odpytaniu modelu językowego o wszystkie miejsca jakie przygotujesz w pliku .csv (format który bardzo łatwo wyeksportujez z/do excela [pomoc](https://support.microsoft.com/en-us/office/import-or-export-text-txt-or-csv-files-5250ac4c-663c-47ce-937b-339e391393ba)).

Do poprawnego działania będzie potrzebny klucz do API OpenAI - [tutorial](https://www.youtube.com/watch?v=SzPE_AE0eEo). Koszt użycia tego generatora to na pewno < 1$.

Żeby uruchomić program musisz kliknąć symbol startu obok komórek z programem. Wykonaj je **PO KOLEI**.

In [1]:
#@title Inicjalizacja programu
!pip install langchain -q

from google.colab import files
import pandas as pd
import io
import getpass
import os

from langchain.chat_models import init_chat_model
from langchain_core.prompts import PromptTemplate

import ipywidgets as widgets
from IPython.display import display

import re

In [2]:
#@title Wgraj swój plik .csv
#@markdown Musi on zawierać kolumnę "Miejsce" i jeżeli generujesz same pytania kolumnę "Opis". Wielkość liter ma znaczenie. Separator ustaw na ";" (domyślnie w Excel).
uploaded = files.upload()

if not uploaded:
    raise ValueError("Please upload a file before continuing.")

filename = list(uploaded.keys())[0]

if not filename.lower().endswith('.csv'):
    raise ValueError("The uploaded file is not a CSV file. Please upload a .csv file.")

try:
    df = pd.read_csv(io.BytesIO(uploaded[filename]), sep=';')
except Exception as e:
    raise ValueError("Could not read CSV with `;` separator. Please check your file format.") from e

if 'Miejsce' not in df.columns:
    raise ValueError("The file does not contain column \"Miejsce\"!")

df.drop(df.columns.difference(['Miejsce', 'Opis']), axis=1, inplace=True)
print(f"✅ Successfully loaded '{filename}' with {df.shape[0]} rows and {df.shape[1]} columns.")
df.head()

Saving Opisy do punktów(Sheet1).csv to Opisy do punktów(Sheet1) (4).csv
✅ Successfully loaded 'Opisy do punktów(Sheet1) (4).csv' with 3 rows and 1 columns.


,Miejsce
0,Tablica wildeckich harcerzy poległych w latach...
1,Tablica pamięci hm. Tadeusza Zielińskiego
2,Pomnik pamięci harcerzy 7 HH


In [3]:
#@title Wprowadź klucz do OpenAI i wybierz model
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

model_name = "openai:gpt-5-nano" # @param {"type":"string"}
llm = init_chat_model(model_name, max_tokens=8192)

Enter your OpenAI API key: ··········


## Generowanie opisów miejsc
Jeżeli chcesz wygenerować same pytania pomiń tę część programu

In [4]:
#@title Ustaw prompt wykorzystywany do generowania opisów miejsc
#@markdown Prompt musi zawierać {place} - tam program wstawi kolejne miejsca. Polecam podać mu przykłady - tak jak w przykładowym prompcie z poprzedniej edycji.
place_prompt_text = """Tworzymy harcerską grę miejską po Poznaniu - mieście w Polsce, stolicy Wielkopolski.
W tym roku styl gry miejskiej to harcerstwo w Poznaniu. Twoim zadaniem jest wygenerować opis miejsca na punkt.
Musi on nawiązywać do tego miejsca i być jakoś związany z harcerstwem, historią harcerstwa w Poznaniu, osobą związaną z harcerstwem i tym miejscem lub Poznańską drużyną harcerską działającą w tamtym miejscu.
Przykłady:
Miejsce: Pomnik pamięci harcerzy 7 HH
Opis: Pomnik znajduje się obok harcówki Hufca Poznań- Jeżyce na Widnej 3. Jest to kompozycja obeliska położonego na poziomej płycie w kształcie prostokąta. Po lewej stronie budowli znajduje się czarna płyta, na której są wypisane nazwiska 31 harcerzy, poległych w obronie ojczyzny w czasie II wojny światowej, lub pomordowanych w hitlerowskich więzieniach i obozach koncentracyjnych. Czarna płyta jest jedynym elementem oryginalnego pomnika, który został postawiony w 1946 pod uwczesną harcówką VII HH przy ul. Grudzieniec 54/58. Odsłonięcie obeliska, który stoi do dnia dzisiejszego, odbyło się 30 kwietnia 1971 roku.
Miejsce: Pomnik pamięci dh Henryka Wysockiego i Edwarda Pietrzykowskiego
Opis: Henryk Wysocki HO urodził się 23 maja 1922 roku w Poznaniu. Był członkiem VI Hufca Harcerzy na Jeżycach i uczniem Szkoły Handlowej w Poznaniu. Edward Pietrzykowski ćwik, urodzony 3 października 1922 roku w Poznaniu, był zastępowym w 2 Poznańskiej Drużynie Harcerskiej i uczniem szkoły ślusarskiej. 1 września 1939 obaj harcerze pełnili służbę obserwacyjno- meldunkową na ulicy Bukowskiej. Zginęli podczas pierwszego bombardowania miasta miasta przez niemieckie samoloty.
Wygeneruj opis dla miejsca: {place}
Opis:
"""
place_prompt_textarea = widgets.Textarea(
    value=place_prompt_text,
    placeholder='Prompt',
    description='Prompt:',
    layout=widgets.Layout(width='80%', height='400px'),
    disabled=False
)

display(place_prompt_textarea)

Textarea(value='Tworzymy harcerską grę miejską po Poznaniu - mieście w Polsce, stolicy Wielkopolski.\nW tym ro…

In [5]:
#@title Generowanie opisów miejsc
#@markdown Program wygeneruje opisy do załadopwnaych miejsc przy użyciu ustawionego prompta.

#@markdown Uruchomienie programu generuje (niewielkie) koszty.

def trim_description_key(response: str) -> str:
  description_match = re.search(r'Opis:\s*(.*)', response, re.DOTALL)
  return description_match.group(1).strip() if description_match else response

place_prompt = PromptTemplate.from_template(place_prompt_textarea.value)
place_chain = place_prompt | llm

response = place_chain.batch(list(df["Miejsce"]))

df = pd.concat([df, pd.DataFrame({"Opis": [trim_description_key(x.content)
                                              for x in response]})], axis=1)
df.head()

,Miejsce,Opis
0,Tablica wildeckich harcerzy poległych w latach...,Tablica upamiętnia wildeckich harcerzy – człon...
1,Tablica pamięci hm. Tadeusza Zielińskiego,Tablica pamięci hm. Tadeusza Zielińskiego zost...
2,Pomnik pamięci harcerzy 7 HH,Pomnik stoi na skwerze przy dawnej harcówce 7....


In [6]:
#@title Zapisz i pobierz Miejsca i Opisy
#@markdown Możesz pominąć ten krok i pobrać wszystko po wygenerowaniu pytań.
filename = "miejsca_i_opisy.csv" #@param {type:"string"}
df.to_csv(filename, index=False)
files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Generowanie pytań
Upewnij się, że masz już w swoich danych zarówno kolumnę "Miejsce" jak i "Opis" (wygenerowane lub zaimportowane).

In [6]:
#@title Ustaw prompt wykorzystywany do generowania pytań do opisów
#@markdown Prompt musi zawierać {place} i {description} - tam program wstawi kolejne miejsca i opisy. Polecam podać mu przykłady - tak jak w przykładowym prompcie z poprzedniej edycji.
question_prompt_text = """Tworzymy harcerską grę miejską po Poznaniu - mieście w Polsce, stolicy Wielkopolski.
Twoim zadaniem jest przygotować pytanie i odpowiedzi do opisu miejsca na które będą odpowiadać uczestnicy gry.Pytanie nie powinno być trudne, oraz nie powinno wymagać żadnej wiedzy poza informacjami z opisu. Zwróć pytanie, poprawną odpowiedź i 3 błędne odpowiedzi w formie ustruktyryzowanej, jak w przykładach.
Przykłady:
Miejsce: Pomnik pamięci harcerzy 7 HH
Opis: Pomnik znajduje się obok harcówki Hufca Poznań- Jeżyce na Widnej 3. Jest to kompozycja obeliska położonego na poziomej płycie w kształcie prostokąta. Po lewej stronie budowli znajduje się czarna płyta, na której są wypisane nazwiska 31 harcerzy, poległych w obronie ojczyzny w czasie II wojny światowej, lub pomordowanych w hitlerowskich więzieniach i obozach koncentracyjnych. Czarna płyta jest jedynym elementem oryginalnego pomnika, który został postawiony w 1946 pod uwczesną harcówką VII HH przy ul. Grudzieniec 54/58. Odsłonięcie obeliska, który stoi do dnia dzisiejszego, odbyło się 30 kwietnia 1971 roku.
Odpowiedź:
question: Co w obecnym pomniku Pamięci Harcerzy 7 HH jest pozostałością po oryginalnym miejscu pamięci?
correct: Czarna płyta z listą nazwisk
incorrect1: Obelisk
incorrect2: Marmurowy prostokąt, na którym stoi obelisk
incorrect3: Płyta znajdująca się na przodzie obeliska
Miejsce: Pomnik pamięci dh Henryka Wysockiego i Edwarda Pietrzykowskiego
Opis: Henryk Wysocki HO urodził się 23 maja 1922 roku w Poznaniu. Był członkiem VI Hufca Harcerzy na Jeżycach i uczniem Szkoły Handlowej w Poznaniu. Edward Pietrzykowski ćwik, urodzony 3 października 1922 roku w Poznaniu, był zastępowym w 2 Poznańskiej Drużynie Harcerskiej i uczniem szkoły ślusarskiej. 1 września 1939 obaj harcerze pełnili służbę obserwacyjno- meldunkową na ulicy Bukowskiej. Zginęli podczas pierwszego bombardowania miasta miasta przez niemieckie samoloty.
Odpowiedź:
question: Co robili druhowie Henryk Wysocki HO i Edward Pietrzykowski Ćwik 1 września 1939?
correct: Pełnili służbę obserwacyjno-meldunkową w Poznaniu
incorrect1: Działali w Pogotowiu Harcerzy
incorrect2: Pojechali na obronę Warszawy
incorrect3: Brali udział w tajnym zebraniu harcerskim
Wygeneruj pytanie dla miejsca:
Miejsce: {place}
Opis: {description}
Odpowiedź:
"""

question_prompt_textarea = widgets.Textarea(
    value=question_prompt_text,
    placeholder='Prompt',
    description='Prompt:',
    layout=widgets.Layout(width='80%', height='400px'),
    disabled=False
)

display(question_prompt_textarea)

Textarea(value='Tworzymy harcerską grę miejską po Poznaniu - mieście w Polsce, stolicy Wielkopolski.\nTwoim za…

In [7]:
#@title Generowanie pytań i odpowiedzi
#@markdown Program wygeneruje pytania do załadopwnaych miejsc i opisów przy użyciu ustawionego prompta.

#@markdown Uruchomienie programu generuje (niewielkie) koszty.
question_prompt = PromptTemplate.from_template(question_prompt_textarea.value)
question_chain = question_prompt | llm

def parse_question_output(response: str) -> tuple:
    question_match = re.search(r'question: (.*)', response)
    correct_match = re.search(r'correct: (.*)', response)
    incorrect1_match = re.search(r'incorrect1: (.*)', response)
    incorrect2_match = re.search(r'incorrect2: (.*)', response)
    incorrect3_match = re.search(r'incorrect3: (.*)', response)

    question = question_match.group(1).strip() if question_match else None
    correct = correct_match.group(1).strip() if correct_match else None
    incorrect1 = incorrect1_match.group(1).strip() if incorrect1_match else None
    incorrect2 = incorrect2_match.group(1).strip() if incorrect2_match else None
    incorrect3 = incorrect3_match.group(1).strip() if incorrect3_match else None

    return question, correct, incorrect1, incorrect2, incorrect3


responses = question_chain.batch([{"place": row["Miejsce"], "description": row["Opis"]}
                      for index, row in df.iterrows()])

questions_data = []
for response in responses:
    question, correct, incorrect1, incorrect2, incorrect3 = parse_question_output(response.content)

    questions_data.append({
        'Pytanie': question,
        'Poprawna odpowiedź': correct,
        'Błędna 1': incorrect1,
        'Błędna 2': incorrect2,
        'Błędna 3': incorrect3
    })

df_questions = pd.DataFrame(questions_data)
df = pd.concat([df, df_questions], axis=1)

df.head()

,Miejsce,Opis,Pytanie,Poprawna odpowiedź,Błędna 1,Błędna 2,Błędna 3
0,Tablica wildeckich harcerzy poległych w latach...,Tablica upamiętnia wildeckich harcerzy – człon...,"Co należy odczytać z tablicy, aby otrzymać has...",Pierwsze litery nazwisk,Daty ich śmierci,Herb hufca,Inskrypcja o odwadze i braterstwie
1,Tablica pamięci hm. Tadeusza Zielińskiego,Tablica pamięci hm. Tadeusza Zielińskiego zost...,"Jakie funkcje pełnił hm. Tadeusz Zieliński, o ...",Drużynowego i instruktora,Harcmistrza i instruktora,Tylko drużynowego,Nauczyciela harcerskiego
2,Pomnik pamięci harcerzy 7 HH,Pomnik stoi na skwerze przy dawnej harcówce 7....,Jaki element jest centralnym na tym pomniku?,"Obelisk z naturalnego kamienia, zwieńczony sty...",Czarna płyta z listami nazwisk,Figura harcerskiego druha trzymającego chorągiew,Płaskorzeźba przedstawiająca symbol ZHP na fro...


In [8]:
#@title Zapisz i pobierz Miejsca i Opisy
#@markdown Możesz pominąć ten krok i pobrać wszystko po wygenerowaniu pytań.
filename = "opisy_i_pytania.csv" #@param {type:"string"}
df.to_csv(filename, index=False)
files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>